# Задачи
1. Предскажем как просто направленности свечи, так и ее величину. В каких-то ситуациях важен тренд, а не конкретное изменение, поэтому предсказания в виде 0 или 1 тоже будет полезно и скорее всего более точным, нежели точное изменение 
2. Обучение будем проводить на основании линейной регрессии, линейной классификации и логистической регресии.

# Машинное обучение

Разделим данные на признаки и целевую переменную

In [111]:
# Нужные пакеты
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

In [102]:
X_lr = X.drop(columns=['target', 'target_trend'])
y = X['target']

Для начала закодируем катеориальные признаки, используя OHE

In [103]:
categorical_features = [col for col in X.columns if 'weekday' in col or 'colour' in col]

X_lr = X.drop(columns=['target', 'target_trend'])
y = X['target']

X_lr = pd.get_dummies(X_lr, columns=categorical_features)

Проведём теперь предобработку данных для машинного обучения, разделим выборку на тренировочную и тестовую, затем проведём нормализацию данных для того, чтобы признаки по равному влияли на вывод модели.
Мы решили использовать именно MinMaxScaler, потому что он сохраняет распределение данных, а только переводит данные в заданный диапазон.

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X_lr, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Перейдём теперь к самому интересному - машинному обучению, для начала обучим обычную модель линейной регрессии и оценим её результаты.

In [105]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

y_pred_train=lr.predict(X_train_scaled)
y_pred = lr.predict(X_test_scaled)

Обучив, обычную линейную модель мы решили использовать,метрики качества такие как MAE,MSE и R2. \
MAE и MSE - дают представление о точности предсказаний модели, но с разным акцентом. MAE показывает среднюю ошибку, а MSE выделяет крупные ошибки, что позволяет получить более полное представление о поведении модели. \
R²-коэффициент детерминации показывает, насколько хорошо модель объясняет дисперсию целевой переменной, что полезно для общей оценки качества модели. \
MAPE мы решили не использовать, поскольку иногда у нас бывает, что целевая перменная равно 0 и из-за этого MAPE мог бы получаться слишком огромным и нерепрезентативным.

In [106]:
mse = mean_squared_error(y_train, y_pred_train)
mae = mean_absolute_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)
mape=mean_absolute_percentage_error(y_train,y_pred_train)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R2",r2)

Mean Squared Error (MSE): 2.398452151717339e-07
Mean Absolute Error (MAE): 0.0002779115913597876
R2 0.7342666925858948


На тренировочных данных достаточно хорошие получились результаты, однако гораздо важнее проверить модель на тестовых данных.

In [107]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape=mean_absolute_percentage_error(y_test,y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R2",r2)

Mean Squared Error (MSE): 2.5933575040877824e-07
Mean Absolute Error (MAE): 0.00028181756538998317
R2 0.7303753643522173


Маленькие значения MSE и MAE указывают на высокую точность модели, однако стоит понимать, что MSE такой низкий не из-за того, что модель предсказывает небольшие величины и если возвести дробь в квадрат, то дробь станет ещё меньше. \
Высокое значение R² (0.73) указывает на то, что модель объясняет значительную часть вариации в данных. В контексте предсказания изменений стоимости акций, где данные могут быть очень шумными и сложными для предсказания, это хороший результат.

Попробуем теперь провести регуляризацию L1 с использованием Лассо. Лассо полезен особенно для нашей модели, у нас поскольку множество признаков,Лассо поможет отобрать признаки, уменьшая некоторые коэффициенты до нуля. Лассо добавляет штраф за абсолютное значение коэффициентов, что приводит к тому, что незначимые признаки могут быть исключены из модели. Это помогает улучшить обобщающую способность модели и уменьшить переобучение.

In [108]:
from sklearn.linear_model import LassoCV
lasso_cv = LassoCV(cv=5, random_state=42)
lasso_cv.fit(X_train_scaled, y_train)

best_alpha = lasso_cv.alpha_

lasso = Lasso(alpha=best_alpha)
lasso.fit(X_train_scaled, y_train)

y_pred = lasso.predict(X_test_scaled)

mse_lasso_cv = mean_squared_error(y_test, y_pred)
mae_lasso_cv = mean_absolute_error(y_test, y_pred)
r2_lasso_cv = r2_score(y_test, y_pred)
mape_lasso_cv = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse_lasso_cv)
print("Mean Absolute Error (MAE):", mae_lasso_cv)
print("R2:", r2_lasso_cv)

Mean Squared Error (MSE): 2.81693546112782e-07
Mean Absolute Error (MAE): 0.0002873343370107747
R2: 0.7071305455754864


Качество модели слегка упало, видимо из-за специфики, Лассо регуляризации некоторые коэффициент стали равны 0, из-за чего модель потеряли некоторые важные взаимосвязи модели и качество ухудшилосью

Интересно построить модель на схожести свечей с другими, поскольку явно то, что было в прошлом могло повториться в будущем и быть может это поможет нам делать более точные прогнозы. Возможно,другой метод машинного обучения KNN будет эффективно использовать исторические данные для предсказания будущих значений, основываясь на схожести текущих условий с прошлыми.

In [112]:
from sklearn.neighbors import KNeighborsRegressor
k_range = range(1, 10)
k_scores = []

for k in k_range:
    knn = KNeighborsRegressor(n_neighbors=k)
    scores = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
    k_scores.append(scores.mean())

optimal_k = k_range[np.argmax(k_scores)]
print(f"Optimal value of k: {optimal_k}")

knn = KNeighborsRegressor(n_neighbors=optimal_k)
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("KNN Regression:")
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²):", r2)

Optimal value of k: 9
KNN Regression:
Mean Squared Error (MSE): 5.671720326635939e-07
Mean Absolute Error (MAE): 0.00044100121355314435
R-squared (R²): 0.4103259870053092


# LogReg для CRS
CRS - Change0, Rsi0, Slowk0

Некторые пункты в объяснении будут пропущены, так как трактовка схожих строк кода была выше

Теперь оценим не точное измение свечи, а ее направление. Предсказывать будем через логистическую регрессию. 

In [113]:
X_CRS = X[['change0', 'rsi0', 'slowk0']]

scaler = MinMaxScaler()
X_CRS_scaled = scaler.fit_transform(X_CRS)
y_CRS = X['target_trend']

LogReg = LogisticRegression()
X_train, X_test,  y_train, y_test = train_test_split(X_CRS_scaled, y_CRS, test_size=0.2, random_state=34)
LogReg.fit(X_train, y_train)
y_predict = LogReg.predict(X_test)

mse = mean_squared_error(y_predict, y_test)
precision = precision_score(y_predict, y_test)
recall = recall_score(y_predict, y_test)

print('mse=', mse)
print("Precision:", precision)
print("Recall:", recall)

mse= 0.29977341389728096
Precision: 0.72829674271775
Recall: 0.70976087260523


Предсказания с ограниченным количеством параметров, которые были выбраны, исходя из графиков и интерпретаций выше, достаточно хороши, но нужно проверить предсказания и со всей имеющейся информацией

# LogReg для all
all - all columns execpt of ['target', 'target_trend']


In [114]:

categorical_features = [col for col in X.columns if 'weekday' in col or 'colour' in col]

X_all = X.drop(columns=['target', 'target_trend'])
y = X['target']

X_all = pd.get_dummies(X_all, columns=categorical_features)

scaler = MinMaxScaler()
X_all_scaled = scaler.fit_transform(X_all)
y_all = X['target_trend']

LogReg = LogisticRegression()
X_train, X_test,  y_train, y_test = train_test_split(X_all_scaled, y_all, test_size=0.2, random_state=34)
LogReg.fit(X_train, y_train)
y_predict = LogReg.predict(X_test)

mse = mean_squared_error(y_predict, y_test)
precision = precision_score(y_predict, y_test)
recall = recall_score(y_predict, y_test)

print('mse=', mse)
print("Precision:", precision)
print("Recall:", recall)


mse= 0.16975075528700906
Precision: 0.8439517864829962
Recall: 0.8352623730739189


Как мы видим предсказания на модели с большим количеством параметров происходят лучше, что вполне ожидаемо.

# Вывод и заключение проекта

По ходу проекта мы спарсили котировки с MOEX. После мы расширили данные новыми колонками. Полученные дата фреймы были предварительно обработали, тажке мы применили созданные функции для агрегации новых показателей по типы "change_all". 

Новые колонки со свечными паттернами в датафрейми объснили в отдельном файле.

Также на основе анализа графиков котировок и иных спомогательных графиков были выведены уже итогвые графики, которые в последствии были интепретированы. 

После анализа данных в разных его проявлениях были выдвинуты гипотезы, которые статистически проверены или опровергнуты. 

Под конец проекта были поставлены задачи на ML, и, основываясь на них, было проведено несколько типов машинного обучения в зависимости от требуемых задач и введенных предпосылок

Итоги каждого из подэтапов последовательно выписывались с прилагающейся интепретацией. Отдельно хочет отметить реалистичный результат каждого из видов машинного обучения, которые описаны выше